Попробуйте обучить нейронную сеть GRU/LSTM для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

In [1]:
import pandas as pd
import numpy as np

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [2]:
import re

In [3]:
df_train = pd.read_csv("C:/Users/kasal/Desktop/gb/14. Фреймворк PyTorch для разработки искусственных нейронных сетей/6/train.csv")
df_test = pd.read_csv("C:/Users/kasal/Desktop/gb/14. Фреймворк PyTorch для разработки искусственных нейронных сетей/6/test.csv")

In [4]:
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
text_ = 'tweet'
class_ = 'label'

In [6]:
sw = set(get_stop_words("ru"))
puncts = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [7]:
df_train[text_] = df_train[text_].apply(preprocess_text)  # re
df_test[text_] = df_test[text_].apply(preprocess_text)

In [8]:
train_corpus = " ".join(df_train[text_])
train_corpus = train_corpus.lower()

In [9]:
# nltk.download("punkt")

tokens = word_tokenize(train_corpus)

In [10]:
max_words = 2000
max_len = 20
num_classes = 1

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [11]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [12]:
# Формируем max_len токенов по частоте 
# Если < max_len то добиваем до max_len

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [13]:
tokens_filtered_top[:10]

['user', 'the', 'to', 'a', 'i', 'you', 'and', 'in', 'for', 'is']

In [14]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [15]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [16]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train[text_]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test[text_]], dtype=np.int32)

CPU times: total: 5.12 s
Wall time: 5.54 s


In [17]:
x_train.shape

(31962, 20)

In [18]:
x_train[1]

array([   1,    1,  170,    9,    5,   64,  426,  620,   70,   68, 1459,
          8,    0,    0,    0,    0,    0,    0,    0,    0])

In [19]:
class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [20]:
train_dataset = DataWrapper(x_train, df_train[class_].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# val_dataset = DataWrapper(x_val, df_val[class_].values)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

### LSTM

(RNN и GRU в методичке)

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [22]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, ht = self.lstm(x)
       
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        # print(out.shape)
        return torch.sigmoid(out)

In [23]:
lstm_init = LSTMFixedLen(max_words, 128, 20, use_last=False)
optimizer = torch.optim.Adam(lstm_init.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [24]:
print(lstm_init)
print("Parameters:", sum([param.nelement() for param in lstm_init.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2000, 128, padding_idx=0)
  (lstm): LSTM(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 271381


In [25]:
lstm_init = lstm_init.to(device)
lstm_init.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    lstm_init.train()
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = lstm_init(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
    # выводим статистику о процессе обучения
    lstm_init.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    # выводим статистику на тестовых данных
#     test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
#     for j, data in enumerate(val_loader):
#         test_labels = data[1].to(device)
#         test_outputs = lstm_init(data[0].to(device))
        
#         # подсчет ошибки на тесте
#         test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
#         # подсчет метрики на тесте
#         test_running_total += len(data[1])
#         pred_test_labels = torch.squeeze((test_outputs > th).int())
#         test_running_right += (test_labels == pred_test_labels).sum()
    
#     test_loss_history.append(test_loss.item())
#     print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
        
print('Training is finished!')

Epoch [1/5]. Step [63/63]. Loss: 0.290. Acc: 0.930. Epoch [2/5]. Step [63/63]. Loss: 0.304. Acc: 0.930. Epoch [3/5]. Step [63/63]. Loss: 0.247. Acc: 0.930. Epoch [4/5]. Step [63/63]. Loss: 0.105. Acc: 0.935. Epoch [5/5]. Step [63/63]. Loss: 0.131. Acc: 0.943. Training is finished!
